In [1]:
from build_dataset import MRILesionDatasetBuilder, LatentImageProcessor, MRIBiomarkemDatasetBuilder

2025-05-23 14:44:07.753891: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-23 14:44:07.776224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748004247.804233 3465216 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748004247.812583 3465216 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748004247.833309 3465216 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Build the dataset for the brain generation model
- Inpaint lesions with simple method in the slices with lesions
- Include SHIFTS and WMH2017 datasets

### VH

In [2]:
data_folder="/home/benet/data"
input_folder="VH"
output_folder="VH-SHIFTS-WMH2017_split"
folders=["train", "test"]
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=0.7
seed = 17844
skip_empty_masks=False
fill_lesion=False

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
dataset_builder.build_dataset()


# output_folder="lesion2D_VH"
# fill_lesion=False
# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()

# output_folder="lesion2D_VH_split"
# fill_lesion=False
# train_split=0.7
# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 741, train examples: 507 (68.42%), test examples: 234 (31.58%)
In the hole dataset: Total examples: 741, train examples: 507 (68.42%), test examples: 234 (31.58%)


### SHIFTS

In [3]:
data_folder="/home/benet/data"
input_folder="SHIFTS_preprocessedMNI" ###
output_folder="VH-SHIFTS-WMH2017_split"
folders=["dev_in", "dev_out", "eval_in", "train"] ###
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=0.7
seed = 17844
skip_empty_masks=False
fill_lesion=False

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
dataset_builder.build_dataset()


# output_folder="lesion2D_SHIFTS"
# fill_lesion=False
# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 1274, train examples: 871 (68.37%), test examples: 403 (31.63%)
In the hole dataset: Total examples: 2015, train examples: 1378 (68.39%), test examples: 637 (31.61%)


### WMH2017

In [4]:
data_folder="/home/benet/data"
input_folder="WMH2017_preprocessedMNI" ###
output_folder="VH-SHIFTS-WMH2017_split"
folders=None ###
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=0.7
seed = 17844
skip_empty_masks=False
fill_lesion=False

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
dataset_builder.build_dataset()


# output_folder="lesion2D_WMH2017"
# fill_lesion=False
# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 780, train examples: 546 (70.00%), test examples: 234 (30.00%)
In the hole dataset: Total examples: 2795, train examples: 1924 (68.84%), test examples: 871 (31.16%)


### LATENTS

In [5]:
from pathlib import Path
import os
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
    print(repo_path)
resolution = 256

# output_dir = "/home/benet/data/lesion2D_VH_split/train/latent"
# input_dir = "/home/benet/data/lesion2D_VH_split/train/flair"
output_dir = "/home/benet/data/VH-SHIFTS-WMH2017_split/train/latent"
input_dir = "/home/benet/data/VH-SHIFTS-WMH2017_split/train/flair"

processor_train = LatentImageProcessor(repo_path, output_dir=output_dir, scale=True, 
                                 resolution=resolution, finetuned_vae=False, input_dir=input_dir)
processor_train.process_images()


# output_dir = "/home/benet/data/lesion2D_VH_split/test/latent"
# input_dir = "/home/benet/data/lesion2D_VH_split/test/flair"
output_dir = "/home/benet/data/VH-SHIFTS-WMH2017_split/test/latent"
input_dir = "/home/benet/data/VH-SHIFTS-WMH2017_split/test/flair"
processor_test = LatentImageProcessor(repo_path, output_dir=output_dir, scale=True,
                                    resolution=resolution, finetuned_vae=False, input_dir=input_dir)
processor_test.process_images()

/home/benet/tfg
Device: cuda
Found 1924 images in the dataset folder
7696 latent images saved in /home/benet/data/VH-SHIFTS-WMH2017_split/train/latent
Device: cuda
Found 871 images in the dataset folder
3484 latent images saved in /home/benet/data/VH-SHIFTS-WMH2017_split/test/latent


## For the model of scanner prompt

In [4]:
data_folder="/home/benet/data"
input_folder="WMH2017_preprocessedMNI" ###
output_folder="WMH2017_split"
folders=None ###
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=1
seed = 17844
skip_empty_masks=False
fill_lesion=False

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
dataset_builder.build_dataset()


Total empty masks skipped: 0
In the preprocessed folder: Total examples: 780, train examples: 780 (100.00%), test examples: 0 (0.00%)
In the hole dataset: Total examples: 780, train examples: 780 (100.00%), test examples: 0 (0.00%)


Manually divide into train/test.
Test: 
- Philips: 35, 37, 39, 41, 49
- Simens: 65, 66, 67, 68, 69
- GE: 116, 126, 132, 137, 144

Now the biomarkem data

In [2]:
slice_mapping = {
    "01": {"ISI": 24, "TSI": 24, "VSI": 27},
    "02": {"ISI": 25, "TSI": 23, "VSI": 26},
    "03": {"ISI": 25, "TSI": 24, "VSI": 25},
    "04": {"ISI": 24, "TSI": 23, "VSI": 24},
    "05": {"ISI": 25, "TSI": 23, "VSI": 26},
    "06": {"ISI": 25, "TSI": 22, "VSI": 26},
    "07": {"ISI": 25, "TSI": 23, "VSI": 24},
    "08": {"ISI": 24, "TSI": 23, "VSI": 24},
    "09": {"ISI": 26, "TSI": 24, "VSI": 26},
    "10": {"ISI": 26, "TSI": 24, "VSI": 25},
    "11": {"ISI": 25, "TSI": 22, "VSI": 25},
    "12": {"ISI": 25, "TSI": 23, "VSI": 25},
    "13": {"ISI": 25, "TSI": 23, "VSI": 24},
    "14": {"ISI": 25, "TSI": 22, "VSI": 26},
    "15": {"ISI": 26, "TSI": 24, "VSI": 25},
}
output_folder="/home/benet/data/WMH2017_split/biomarkem"
builder = MRIBiomarkemDatasetBuilder(slice_mapping=slice_mapping, output_folder=output_folder)
builder.build_dataset()


Manually divide into train/test.
Test: 11, 12, 13, 14, 15

Now, create the latents

In [3]:
from pathlib import Path
import os
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
    print(repo_path)
resolution = 256

output_dir = "/home/benet/data/WMH2017_split/train/latent"
input_dir = "/home/benet/data/WMH2017_split/train/flair"

processor_train = LatentImageProcessor(repo_path, output_dir=output_dir, scale=True, 
                                 resolution=resolution, finetuned_vae=False, input_dir=input_dir)
processor_train.process_images()

output_dir = "/home/benet/data/WMH2017_split/test/latent"
input_dir = "/home/benet/data/WMH2017_split/test/flair"
processor_test = LatentImageProcessor(repo_path, output_dir=output_dir, scale=True,
                                    resolution=resolution, finetuned_vae=False, input_dir=input_dir)
processor_test.process_images()

/home/benet/tfg
Device: cuda
Found 735 images in the dataset folder
2940 latent images saved in /home/benet/data/WMH2017_split/train/latent
Device: cuda
Found 270 images in the dataset folder
1080 latent images saved in /home/benet/data/WMH2017_split/test/latent
